In [35]:
import google.generativeai as genai
import os
from typing import Union, List
import dotenv

In [36]:
# Load API key from environment variables
dotenv.load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please set the 'GOOGLE_API_KEY' environment variable.")

genai.configure(api_key=api_key)

In [37]:
class PromptTemplates:
    @staticmethod
    def get_language_detection_prompt(text: str) -> str:
        examples = """
        Example inputs and outputs:
        Text: "Hello world" -> en
        Text: "Bonjour le monde" -> fr
        Text: "Xin chào thế giới" -> vi
        Text: "こんにちは世界" -> ja
        
        Now detect the language for this text and respond only with the ISO 639-1 code:
        """
        return f"{examples} {text}"

    @staticmethod
    def get_translation_prompt(text: str, target_lang: str) -> str:
        examples = f"""
        Example translations:
        Input: "Hello", Target: es -> "Hola"
        Input: "Good morning", Target: fr -> "Bonjour"
        Input: "Thank you", Target: vi -> "Cảm ơn"
        
        Translate this text into {target_lang} only responding with the translated text: 
        """
        return f"{examples} {text}"

class TranslationService:
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(
            model_name="gemini-1.5-flash",
            generation_config={
                "temperature": 1,
                "top_p": 0.95,
                "top_k": 40,
                "max_output_tokens": 8192,
                "response_mime_type": "text/plain",
            },
        )
        self.prompt_templates = PromptTemplates()

    def detect_language(self, text: str) -> str:
        prompt = self.prompt_templates.get_language_detection_prompt(text)
        response = self.model.generate_content(prompt)
        return response.text.strip().lower()

    def translate(self, text: str, target_lang: str) -> str:
        source_lang = self.detect_language(text)
        if source_lang == target_lang:
            return text
            
        prompt = self.prompt_templates.get_translation_prompt(text, target_lang)
        response = self.model.generate_content(prompt)
        return response.text.strip()

In [38]:
def translate_single_text(input_json: dict) -> str:
    text = input_json.get('text', '')
    if not isinstance(text, str):
        raise ValueError("Text must be a string")

    language = input_json.get('dest_language', '')
    translator = TranslationService(api_key)
    return translator.translate(text, language)

def translate_multiple_texts(input_json: dict) -> List[str]:
    texts = input_json.get('text', [])
    if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
        raise ValueError("Text must be a list of strings")

    language = input_json.get('dest_language', '')
    translator = TranslationService(api_key)
    return [translator.translate(text, language) for text in texts]

In [39]:
# Ví dụ 1: Dịch một văn bản đơn lẻ
json_1 = {
    'text': 'Hello',
    'dest_language': 'es'
}


translate_single_text(json_1)


'Hola'

In [40]:
# Ví dụ 2: Dịch nhiều văn bản
json_2 = {
    'text': ['Hello', 'I am Peter',' How are you?'],
    'dest_language': 'vi'
}
translate_multiple_texts(json_2)

['Xin chào', 'Tôi là Peter', 'Bạn khỏe không?']

In [41]:
json_3 = {
    'text':['私はあなたを愛しています'],
    'dest_language':'en'
}
translate_multiple_texts(json_3)

['I love you']